In [1]:
!pip install pandas

In [2]:
import torch, baukit
from transformers import AutoModelForCausalLM, AutoTokenizer
import pandas as pd
import numpy as np
from baukit import pbar

In [3]:
#MODEL_NAME = "gpt2-xl"  # gpt2-xl or EleutherAI/gpt-j-6B
MODEL_NAME = "gpt2-xl"
model, tok = (
    AutoModelForCausalLM.from_pretrained(MODEL_NAME, low_cpu_mem_usage=False).to("cuda"),
    AutoTokenizer.from_pretrained(MODEL_NAME)
)
baukit.set_requires_grad(False, model)

In [4]:
import baukit
baukit.set_requires_grad(False, model)

In [5]:
def get_key_vectors(model, tok, tokens, layer_names = None):
    import re
    from baukit import TraceDict
    inp = {k: torch.tensor(v)[None].cuda() for k, v in tok(tokens).items()}
    if layer_names is None:
        layer_names = [n for n,_ in model.named_modules() if 'mlp.c_proj' in n]
    with TraceDict(model, layer_names, retain_input=True) as tr:
        logits = model(**inp)['logits']
    return np.stack([tr[layername].input[0][-1].cpu().detach().numpy() for layername in layer_names])

tokens = 'emma watson'
hs = get_key_vectors(model, tok, tokens)
hs.shape
type(hs)

numpy.ndarray

In [6]:
token_csv = "../data/tokens.csv"
token_df = pd.read_csv(token_csv)
tokens = token_df['token'].values

In [7]:
npy_datapath = "../data/npy_files"
mlp_proj_layers = [n for n,_ in model.named_modules() if 'mlp.c_proj' in n]
npy_filepaths = [npy_datapath + "/" + "-".join(n.split('.')[:3]) + ".npy"
                 for n in mlp_proj_layers]

In [8]:
for npy_file, layer_name in pbar(zip(npy_filepaths, mlp_proj_layers)):
    final_data = []
    for i in range(len(tokens)):
        data = get_key_vectors(model,tok,tokens[i],[layer_name])
        data = data.reshape(-1)
        final_data.append(data)
    final_data = np.array(final_data)
    np.save(npy_file, final_data)

0it [00:00, ?it/s]